In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [2]:
totals_words = 10000#只使用最常用的前10000个单词
embedding_len = 100#每个单词的维度为100
max_review_len = 80#每个句子的长度
batch_size = 128
(x_train,y_train),(x_val,y_val) = keras.datasets.imdb.load_data(num_words=totals_words)#只使用最常用的10000个单词，其他单词标记为未知
x_train = keras.preprocessing.sequence.pad_sequences(x_train,maxlen=max_review_len)#设置每个句子的长度为80
x_val = keras.preprocessing.sequence.pad_sequences(x_val,maxlen=max_review_len)
db_train = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(batch_size,drop_remainder=True)
db_val = tf.data.Dataset.from_tensor_slices((x_val,y_val)).shuffle(1000).batch(batch_size,drop_remainder=True)#最后一个batch长度不够将其drop

db_iter = iter(db_train)
sample = next(db_iter)
print(x_train.shape,y_train.shape,tf.reduce_max(y_train),tf.reduce_min(y_train))
print(x_val.shape,y_val.shape)
print(sample[0].shape)
print(sample[1].shape)

(25000, 80) (25000,) tf.Tensor(1, shape=(), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
(25000, 80) (25000,)
(128, 80)
(128,)


In [3]:
class MyRnn(keras.Model):
    def __init__(self,units):
        super(MyRnn,self).__init__()
        #[b,80]=>[b,80,100]
        self.embedding = layers.Embedding(input_dim=totals_words,output_dim=embedding_len,input_length=max_review_len)
        
        #[b,80]=>[b,units]
        self.rnn_cell0 = layers.GRU(units,return_sequences=True,dropout=0.5,unroll=True)#返回输出序列中的最后一个输出
        self.rnn_cell1 = layers.GRU(units,dropout=0.5,unroll=True)#unroll 是否展开网络
        #[b,units]=>[b,1]
        self.fc = layers.Dense(1)
        
    def call(self,inputs,training=None):
        #x = [b,80]
        x = inputs
        #[b,80]=>[b,80,100]
        x = self.embedding(x)
        #[b,80]=>[b,64]
        x = self.rnn_cell0(x)
        x = self.rnn_cell1(x)
        #[b,64]=>[b,1]
        out = self.fc(x)
        #p(y is positive|x)
        pred = tf.sigmoid(out)
        
        return pred

In [4]:
def main():
    units = 64
    model = MyRnn(units)
    model.compile(optimizer=keras.optimizers.Adam(0.0002),loss=tf.losses.BinaryCrossentropy(),metrics=['accuracy'])
    model.fit(db_train,epochs=5,validation_data=db_val)
main()

W0825 01:15:42.753564 10196 deprecation.py:506] From D:\Anaconda3\lib\site-packages\tensorflow\python\keras\backend.py:4081: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Epoch 1/5
195/195 [==============================] - 147s 751ms/step - loss: 0.6604 - accuracy: 0.5398 - val_loss: 0.4896 - val_accuracy: 0.7607
Epoch 2/5
195/195 [==============================] - 48s 245ms/step - loss: 0.4318 - accuracy: 0.7755 - val_loss: 0.3762 - val_accuracy: 0.8334
Epoch 3/5
195/195 [==============================] - 47s 242ms/step - loss: 0.3443 - accuracy: 0.8503 - val_loss: 0.3618 - val_accuracy: 0.8391
Epoch 4/5
195/195 [==============================] - 47s 240ms/step - loss: 0.3053 - accuracy: 0.8704 - val_loss: 0.3594 - val_accuracy: 0.8404
Epoch 5/5
195/195 [==============================] - 48s 245ms/step - loss: 0.2802 - accuracy: 0.8860 - val_loss: 0.3610 - val_accuracy: 0.8414
